In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import *
from sklearn.metrics.pairwise import *

In [63]:
df = pd.read_excel("IELTS_Success_Stories_Dataset.xlsx")

In [64]:
df.head()

,Candidate,Location,Profession,Study Duration (months),IELTS Score (Overall),Key Strategies,Education Level,Age,Target Country,English Proficiency (Preparation),...,Preferred Learning Resources,Application Status,Job Offer Received,Additional Certifications,Volunteer Experience,Language Fluency,Internship Experience,Relevant Skills,Recommendations,Networking Efforts
0,Aisha,Karachi,Grad Student,3,8.50,"Practice, forums, tutoring",Master's,24,Canada,Advanced,...,"Online courses, study guides",Under Process,No,TESOL,Yes,"Urdu, English",Research Assistant,"Data Analysis, Teaching",Strong,"LinkedIn, Local Events"
1,Ahmed,Lahore,IT Professional,4,7.50,"Balanced schedule, time management, speaking p...",Bachelor's,29,Australia,Intermediate,...,"Online forums, practice tests",Approved,Yes,PMP,No,"Urdu, English",IT Consultant,"Project Management, Programming",Positive,Online Communities
2,Fatima,Islamabad,School Teacher,2,9.00,"Intensive course, online resources, mock inter...",Master's,32,New Zealand,Advanced,...,"Study guides, language exchange",Under Process,No,CELTA,Yes,"Urdu, English",Curriculum Developer,"Lesson Planning, Educational Tech",Positive,Professional Groups
3,Ali,Peshawar,IT Professional,5,8.00,"Focused study, workshops, peer feedback",Bachelor's,27,USA,Advanced,...,"Mock exams, speaking clubs",Rejected,No,AWS Certified Solutions Architect,No,"Urdu, English",Software Developer,"Cloud Computing, Programming",Strong,Online Platforms
4,Saad,Lahore,Eng. Student,6,8.25,"Consistent practice, private tutor, speaking c...",Bachelor's,22,UK,Advanced,...,"Online tutorials, speaking practice",Under Process,No,IELTS Examiner,Yes,"Urdu, English",Research Assistant,"Lab Techniques, Data Analysis",Positive,University Events


In [78]:
def retrieve_no_of_columns_to_split(x):
    pp = []
    ctr = 0
    for i in df[x].unique():
        pp += i.split(",")
        ctr += 1
    qq = []
    for i in pp:
        qq.append(i.strip().lower())
    return len(qq)//ctr

In [79]:
columns_to_split = ["Key Strategies","Preferred Learning Resources"]
columns_to_remove = ["Candidate","Location","Target Country","Application Status","Job Offer Received","Additional Certifications","Language Fluency","Relevant Skills","Age"]

In [80]:
cols = df.columns.values.tolist()
columns_new = []
start = 0
df_1 = df.copy()
for i in columns_to_split:
    data = df[i].copy()
    d = np.array(list(map(lambda x: [i.strip().lower() for i in x.split(", ")],data)))
    ctr = retrieve_no_of_columns_to_split(i)
    new_col = []
    for j in range(ctr):
        ne = i+" "+str(j)
        df_1[ne] = d[:,j]
        new_col.append(ne)
    stop = cols.index(i)
    columns_new += cols[start:stop]
    columns_new += new_col
    start = stop + 1
columns_new += cols[start:]
df_2 = df_1[columns_new].copy()
df_2.drop(columns=columns_to_remove,inplace=True)
df_2["IELTS Score (Overall)"] = df_2["IELTS Score (Overall)"].round(2)
correct_order_col = []
for i in df_2.columns:
    if i != "IELTS Score (Overall)":
        correct_order_col.append(i)
correct_order_col.append("IELTS Score (Overall)")
df_2 = df_2[correct_order_col]

In [81]:
df_2.head()

,Profession,Study Duration (months),Key Strategies 0,Key Strategies 1,Key Strategies 2,Education Level,English Proficiency (Preparation),Practice Hours per Week,Mock Tests Taken,Achieved Desired Score,Preferred Learning Resources 0,Preferred Learning Resources 1,Volunteer Experience,Internship Experience,Recommendations,Networking Efforts,IELTS Score (Overall)
0,Grad Student,3,practice,forums,tutoring,Master's,Advanced,15,5,Yes,online courses,study guides,Yes,Research Assistant,Strong,"LinkedIn, Local Events",8.50
1,IT Professional,4,balanced schedule,time management,speaking practice,Bachelor's,Intermediate,10,3,Yes,online forums,practice tests,No,IT Consultant,Positive,Online Communities,7.50
2,School Teacher,2,intensive course,online resources,mock interviews,Master's,Advanced,20,8,Yes,study guides,language exchange,Yes,Curriculum Developer,Positive,Professional Groups,9.00
3,IT Professional,5,focused study,workshops,peer feedback,Bachelor's,Advanced,12,4,Yes,mock exams,speaking clubs,No,Software Developer,Strong,Online Platforms,8.00
4,Eng. Student,6,consistent practice,private tutor,speaking clubs,Bachelor's,Advanced,18,6,Yes,online tutorials,speaking practice,Yes,Research Assistant,Positive,University Events,8.25


In [90]:
key_strategies_initial = list(set(df_2["Key Strategies 0"].unique().tolist() + df_2["Key Strategies 1"].unique().tolist() + df_2["Key Strategies 2"].unique().tolist()))

preferred_learning_resources_initial = list(set(df_2["Preferred Learning Resources 0"].unique().tolist() + df_2["Preferred Learning Resources 1"].unique().tolist()))

In [95]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(key_strategies_initial)
cosine_matrix = pd.DataFrame(cosine_similarity(tfidf_matrix).round(2),index=key_strategies_initial,columns=key_strategies_initial)
cosine_matrix

,online resources,study groups,forums,focused study,tutoring,speaking with native speakers,balanced schedule,mock tests,group study,self-paced study,...,listening skill improvement,peer feedback,intensive speaking practice,consistent practice,daily practice,intensive course,speaking clubs,online forums,private tutor,structured study plan
online resources,1.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.43,0.0,0.00
study groups,0.00,1.00,0.00,0.29,0.0,0.00,0.0,0.00,0.29,0.23,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.22
forums,0.00,0.00,1.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.74,0.0,0.00
focused study,0.00,0.29,0.00,1.00,0.0,0.00,0.0,0.00,0.29,0.23,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.22
tutoring,0.00,0.00,0.00,0.00,1.0,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
speaking with native speakers,0.00,0.00,0.00,0.00,0.0,1.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.23,0.00,0.00,0.00,0.24,0.00,0.0,0.00
balanced schedule,0.00,0.00,0.00,0.00,0.0,0.00,1.0,0.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
mock tests,0.00,0.00,0.00,0.00,0.0,0.00,0.0,1.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
group study,0.00,0.29,0.00,0.29,0.0,0.00,0.0,0.00,1.00,0.23,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.22
self-paced study,0.00,0.23,0.00,0.23,0.0,0.00,0.0,0.00,0.23,1.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.18


In [32]:
retrieve_no_of_columns_to_split("Preferred Learning Resources")

2